In [60]:
# Get weather data from API call

import requests
import sys

import json
                

response = requests.request("GET", "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/istanbul/2022-01-01/2022-12-31?unitGroup=metric&include=days&key=2C4WD8FM3QA2TW494W2K42DX5&contentType=json")
if response.status_code!=200:
  print('Unexpected Status code: ', response.status_code)
  sys.exit()  


# Parse the results as JSON
jsonData = response.json()

In [62]:
# Create weather data into a dataframe

import pandas as pd

weather_data = []


for i in range(0,365):
    x = jsonData["days"][i]
    weather_data.append(
    {
        "Date": x["datetime"],
        "Temp": x["temp"],
        "Temp Min": x["tempmin"],
        "Temp Max": x["tempmax"],
        "Precipitation": x["precip"],
        "Type of Precip": x["preciptype"],
        "Conditions": x["conditions"],
        "Description": x["description"]
    })
    
weather_data = pd.DataFrame(weather_data)

#weather_data

,Date,Temp,Temp Min,Temp Max,Precipitation,Type of Precip,Conditions,Description
0,2022-01-01,8.8,6.8,10.5,0.303,[rain],"Rain, Partially cloudy",Partly cloudy throughout the day with late aft...
1,2022-01-02,9.6,8.2,10.7,5.932,[rain],"Rain, Partially cloudy",Partly cloudy throughout the day with a chance...
2,2022-01-03,10.0,7.1,12.3,0.000,None,Partially cloudy,Partly cloudy throughout the day.
3,2022-01-04,11.4,9.1,14.1,0.000,None,Partially cloudy,Partly cloudy throughout the day.
4,2022-01-05,13.0,10.0,16.6,0.000,None,Partially cloudy,Partly cloudy throughout the day.
...,...,...,...,...,...,...,...,...
360,2022-12-27,10.8,7.0,13.3,0.000,None,Partially cloudy,Partly cloudy throughout the day.
361,2022-12-28,10.6,8.9,12.5,0.086,[rain],"Rain, Partially cloudy",Partly cloudy throughout the day with late aft...
362,2022-12-29,8.8,6.9,11.8,0.052,[rain],"Rain, Partially cloudy",Clearing in the afternoon with morning rain.
363,2022-12-30,10.4,7.6,13.0,0.000,None,Partially cloudy,Clearing in the afternoon.


In [63]:
# Change weather data date to proper datetime

weather_data['Date'] = pd.to_datetime(weather_data['Date'], format = "%Y-%m-%d")

,Date,Temp,Temp Min,Temp Max,Precipitation,Type of Precip,Conditions,Description
0,2022-01-01,8.8,6.8,10.5,0.303,[rain],"Rain, Partially cloudy",Partly cloudy throughout the day with late aft...
1,2022-01-02,9.6,8.2,10.7,5.932,[rain],"Rain, Partially cloudy",Partly cloudy throughout the day with a chance...
2,2022-01-03,10.0,7.1,12.3,0.000,None,Partially cloudy,Partly cloudy throughout the day.
3,2022-01-04,11.4,9.1,14.1,0.000,None,Partially cloudy,Partly cloudy throughout the day.
4,2022-01-05,13.0,10.0,16.6,0.000,None,Partially cloudy,Partly cloudy throughout the day.


In [40]:
# Extracted and read mall data csv from Kaggle
## https://www.kaggle.com/datasets/mehmettahiraslan/customer-shopping-dataset 

mall = pd.read_csv("C:/Users/ds2002admin/Downloads/archive (1)/customer_shopping_data.csv")

mall["year"] = mall.invoice_date.str[-2:]

In [41]:
# Change mall data dates to proper datetime

mall['invoice_date'] = pd.to_datetime(mall['invoice_date'], format = "%d/%m/%Y")

,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall,year
0,I138884,C241288,Female,28,Clothing,5,1500.40,Credit Card,2022-08-05,Kanyon,22
1,I317333,C111565,Male,21,Shoes,3,1800.51,Debit Card,2021-12-12,Forum Istanbul,21
2,I127801,C266599,Male,20,Clothing,1,300.08,Cash,2021-11-09,Metrocity,21
3,I173702,C988172,Female,66,Shoes,5,3000.85,Credit Card,2021-05-16,Metropol AVM,21
4,I337046,C189076,Female,53,Books,4,60.60,Cash,2021-10-24,Kanyon,21


In [42]:
# Extracted and read customer data csv from Kaggle
## https://www.kaggle.com/datasets/vjchoudhary7/customer-segmentation-tutorial-in-python 

cust = pd.read_csv("C:/Users/ds2002admin/Downloads/archive/Mall_Customers.csv")

,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


In [43]:
# Group by gender and age to create single identities (i.e. unique gender-age pairings)

cgroup = cust.groupby(['Gender', 'Age'])['Annual Income (k$)', 'Spending Score (1-100)'].mean()

In [44]:
cgroup.reset_index(inplace = True)

In [45]:
# Add customer IDs to the customer data

cgroup['customer_id'] = range(len(cgroup))
cgroup.customer_id = cgroup.customer_id+1

# Dropping a column

mall.drop(columns = ['customer_id'], inplace = True)

# Renaming and adjusting columns for both datasets

cgroup.rename({'Gender': 'gender', 'Age': 'age'}, axis=1, inplace=True)
mall2 = pd.merge(mall, cgroup, on = ['gender','age'])
mall2.dropna(inplace = True)
mall2.drop(columns = ['gender', 'age', 'year', 'Annual Income (k$)', 'Spending Score (1-100)'], inplace = True)
mall2.head()

,invoice_no,category,quantity,price,payment_method,invoice_date,shopping_mall,customer_id
0,I138884,Clothing,5,1500.40,Credit Card,2022-08-05,Kanyon,10
1,I227836,Clothing,5,1500.40,Credit Card,2022-05-24,Forum Istanbul,10
2,I236914,Clothing,3,900.24,Cash,2021-10-26,Mall of Istanbul,10
3,I307534,Cosmetics,4,162.64,Cash,2022-03-26,Viaport Outlet,10
4,I720017,Food & Beverage,2,10.46,Credit Card,2022-09-04,Kanyon,10


In [51]:
# Converting mall to csv, then loading the fact table into SQL

mall2.to_csv('mall2.csv', index = False)

In [ ]:
# Creating the database for SQL

import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="ds2002",
  password="UVA!1819"
)

mycursor = mydb.cursor()

mycursor.execute("CREATE DATABASE project2002")

In [53]:
import os
import pandas as pd
import numpy as np

try:
    #data_dir = os.patch.join(os.getcwd(), 'data')
    data_file = 'C:/Users/ds2002admin/Downloads/mall2.csv'
    
    

    df = pd.read_csv(data_file)
    display(df.head())

except:
    print("could not read csv")

,invoice_no,category,quantity,price,payment_method,invoice_date,shopping_mall,customer_id
0,I138884,Clothing,5,1500.40,Credit Card,2022-08-05,Kanyon,10
1,I227836,Clothing,5,1500.40,Credit Card,2022-05-24,Forum Istanbul,10
2,I236914,Clothing,3,900.24,Cash,2021-10-26,Mall of Istanbul,10
3,I307534,Cosmetics,4,162.64,Cash,2022-03-26,Viaport Outlet,10
4,I720017,Food & Beverage,2,10.46,Credit Card,2022-09-04,Kanyon,10


In [54]:
#!pip install PyMySQL
#!pip install mysql-connector-python
#!pip install sqlalchemy
import os
import pymysql
import mysql.connector
from sqlalchemy import create_engine

import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

host_name = "localhost"
host_ip = "127.0.0.1"
port = "3306"

user_id = "ds2002"
pwd = "UVA!1819"
db_name = "project2002"

def insert_sqlalchemy_dataframe(user_id, pwd, host_name, db_name, df, table_name):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    df.to_sql(table_name, con=connection, if_exists='replace') #, index_label='product_id');  'append'
    connection.close()
 #
try:
    insert_sqlalchemy_dataframe(user_id, pwd, host_name, db_name, df, 'mall')
except:
    print("could not convert pandas dataframe to SQL database")


In [56]:
# Converting the customer data to a csv, then creating a dimension table in SQL

cgroup.to_csv('cgroup.csv', index = False)

In [57]:
import os
import pandas as pd
import numpy as np

try:
    #data_dir = os.patch.join(os.getcwd(), 'data')
    data_file = 'C:/Users/ds2002admin/Downloads/cgroup.csv'
    
    

    df = pd.read_csv(data_file)
    display(df.head())

except:
    print("could not read csv")

,gender,age,Annual Income (k$),Spending Score (1-100),customer_id
0,Female,18,65.00,48.00,1
1,Female,19,64.00,52.00,2
2,Female,20,26.50,40.50,3
3,Female,21,44.75,63.25,4
4,Female,22,37.00,65.50,5


In [58]:
import os
import pymysql
import mysql.connector
from sqlalchemy import create_engine

import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

host_name = "localhost"
host_ip = "127.0.0.1"
port = "3306"

user_id = "ds2002"
pwd = "UVA!1819"
db_name = "project2002"

def insert_sqlalchemy_dataframe(user_id, pwd, host_name, db_name, df, table_name):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    df.to_sql(table_name, con=connection, if_exists='replace') #, index_label='product_id');  'append'
    connection.close()
 #
try:
    insert_sqlalchemy_dataframe(user_id, pwd, host_name, db_name, df, 'customers')
except:
    print("could not convert pandas dataframe to SQL database")


In [64]:
# Converting the extracted weather data dataframe into a csv, then reading into SQL as a dimension table

weather_data.to_csv('weather.csv', index = False)

In [66]:
import os
import pandas as pd
import numpy as np

try:
    #data_dir = os.patch.join(os.getcwd(), 'data')
    data_file = 'C:/Users/ds2002admin/Downloads/weather.csv'
    
    

    df = pd.read_csv(data_file)
    display(df.head())

except:
    print("could not read csv")

,Date,Temp,Temp Min,Temp Max,Precipitation,Type of Precip,Conditions,Description
0,2022-01-01,8.8,6.8,10.5,0.303,['rain'],"Rain, Partially cloudy",Partly cloudy throughout the day with late aft...
1,2022-01-02,9.6,8.2,10.7,5.932,['rain'],"Rain, Partially cloudy",Partly cloudy throughout the day with a chance...
2,2022-01-03,10.0,7.1,12.3,0.000,NaN,Partially cloudy,Partly cloudy throughout the day.
3,2022-01-04,11.4,9.1,14.1,0.000,NaN,Partially cloudy,Partly cloudy throughout the day.
4,2022-01-05,13.0,10.0,16.6,0.000,NaN,Partially cloudy,Partly cloudy throughout the day.


In [67]:
import os
import pymysql
import mysql.connector
from sqlalchemy import create_engine

import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

host_name = "localhost"
host_ip = "127.0.0.1"
port = "3306"

user_id = "ds2002"
pwd = "UVA!1819"
db_name = "project2002"

def insert_sqlalchemy_dataframe(user_id, pwd, host_name, db_name, df, table_name):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    df.to_sql(table_name, con=connection, if_exists='replace') #, index_label='product_id');  'append'
    connection.close()
 #
try:
    insert_sqlalchemy_dataframe(user_id, pwd, host_name, db_name, df, 'weather')
except:
    print("could not convert pandas dataframe to SQL database")